In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import gensim
import jamo
try:
    from eunjeon import Mecab
except:
    from konlpy.tag import Mecab
from tqdm.auto import tqdm

tqdm.pandas()

In [2]:
DATA_TYPE = 'full'
MECAB_TYPE = 'morphs' # nouns or morphs

if DATA_TYPE.startswith('outer'):
    TEXT_COLUMNS = ['headline', 'lede']
elif DATA_TYPE.startswith('inner'):
    TEXT_COLUMNS = ['title', 'contents']
else:
    TEXT_COLUMNS = ['title', 'lede', 'contents']

In [3]:
PREPROCESSING_DIR = '2-preprocessing'
TOKENIZING_DIR = '3-tokenizing'
DATA_DIR = 'lol'

preprocessing_dir = Path(PREPROCESSING_DIR) / DATA_DIR
tokenizing_dir = Path(TOKENIZING_DIR) / DATA_DIR
tokenizing_dir.mkdir(parents=True, exist_ok=True)

In [4]:
mecab = Mecab()
if MECAB_TYPE == 'nouns':
    mecab_fn = lambda x: ' '.join(mecab.nouns(x)) if x is not np.nan else x
else:
    mecab_fn = lambda x: ' '.join(mecab.morphs(x)) if x is not np.nan else x

In [6]:
df = pd.read_csv(preprocessing_dir / ('news-%s.csv' % DATA_TYPE))
df.tail()

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
51811,20200430,20200430 17:49,20200430 17:56,26,1,"[LCK 승강전] 다이브 받아치면 곧 승리! 샌드박스, 서라벌에 1세트 선승",30일 종각 LoL 파크에서 열린 열린 2020 LCK 섬머 스플릿 승강전 최종전 ...,인벤,30일 종각 LoL 파크에서 열린 열린 2020 LCK 섬머 스플릿 승강전 최종전 ...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,"[롤챔스 승강전] 서라벌-샌드박스, 베스트 라인업으로 맞대결",LCK 2020 서머 최종전에서 대결하는 서라벌 게이밍(위)과 샌드박스 게이밍(사진...,데일리e스포츠,마지막 승강전에서 승리를 원하는 서라벌 게이밍과 샌드박스 게이밍이 승강전 내내 기용...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,"‘일방적인 전투 승리’…샌드박스게이밍, 서라벌게이밍 상대로 1세트 승리",30일 ‘2020 우리은행 LCK Spring Split’ 승강전 최종진출전에서는 ...,엑스포츠뉴스,30일 ‘2020 우리은행 LCK Spring Split’ 승강전 최종진출전에서는 ...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,"[LCK 승강전] 샌드박스, 절묘한 카운터로 손쉽게 기선 제압",[OSEN=고용준 기자] LCK 최후의 승강전에서 샌드박스가 먼저 웃었다. 다이브 ...,OSEN,LCK 최후의 승강전에서 샌드박스가 먼저 웃었다. 다이브 공세를 절묘하게 받아치면서...,0.0,490
51815,20200430,20200430 18:51,20200430 18:51,30,1,"[LCK 승강전] 샌드박스, 집중력 앞세워 혼전 승리 2-0 리드",[OSEN=고용준 기자] 잡은 기회는 확실하게 놓치지 않았다. 샌드박스가 LCK 최...,OSEN,잡은 기회는 확실하게 놓치지 않았다. 샌드박스가 LCK 최후의 승강전 생환까지 단 ...,0.0,460


In [43]:
title ,= df[(df.date == 20150905) & (df.ranking == 9)].title
print(title)

[롤드컵 선발전]KT, 야스오 앞세운 운영으로 2:0(2세트)


In [47]:
ndf = pd.read_csv('4-embedding/lol/news-full-nouns-jamo-trigram.csv')

In [48]:
print(ndf[(ndf.date == 20150905) & (ndf.ranking == 9)].title)

1003    ㄹㅗㄹ_ㄷㅡ_ㅋㅓㅂ ㅅㅓㄴㅂㅏㄹㅈㅓㄴ ㅇㅑㅅㅡㅇㅗ ㅇㅜㄴㅇㅕㅇ ㅅㅔㅌㅡ
Name: title, dtype: object


In [46]:
print('   '.join(mecab.nouns(title)))

롤   드   컵   선발전   야스오   운영   세트


In [45]:
jamo.j2hcj(jamo.h2j('   '.join(mecab.nouns(title))))

'ㄹㅗㄹ   ㄷㅡ   ㅋㅓㅂ   ㅅㅓㄴㅂㅏㄹㅈㅓㄴ   ㅇㅑㅅㅡㅇㅗ   ㅇㅜㄴㅇㅕㅇ   ㅅㅔㅌㅡ'

In [5]:
mecab_df = df.copy()
for column in TEXT_COLUMNS:
    mecab_df[column] = mecab_df[column].progress_apply(mecab_fn)
mecab_df.to_csv(tokenizing_dir / ('news-%s-%s.csv' % (DATA_TYPE, MECAB_TYPE)), index=False)
mecab_df

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
0,20150801,20150801 00:49,20150801 00:49,1,1,[ 롤 챔스 핫 매치 리뷰 ] ' 불량 학생 ' 노동현 의 수능 대박 스토리,""" 탈선 했 어요 ! """" 수능 만점 ! "" 2015 년 7 월 31 일 , KOO...",인벤,""" 탈선 했 어요 ! "". "" 수능 만점 ! "". 2015 년 7 월 31 일 , ...",0.0,198042
1,20150801,20150801 23:32,20150801 23:32,2,1,"[ 롤 챔스 섬머 ] 스 베누 소닉붐 뉴 클리어 - 사신 인터뷰 , 앞 으로 는 승...",금일 ( 1 일 ) 롤 챔스 섬머 시즌 사상 스 베누 소닉붐 이 첫 번 째 승리 를...,헝그리앱,금일 ( 1 일 ) 롤 챔스 섬머 시즌 사상 스 베누 소닉붐 이 첫 번 째 승리 를...,0.0,8657
2,20150801,20150801 20:10,20150801 20:10,3,1,[ 롤 챔스 섬머 ] 뱅 배준식 인터뷰 . 솔로 랭크 를 올리 는 데 는 멘탈 이 ...,Q . 오늘 승리 한 소감 은 어떻게 되 나 ? 오늘 경기 에 오랜만 에 배성웅 (...,헝그리앱,Q . 오늘 승리 한 소감 은 어떻게 되 나 ? 오늘 경기 에 오랜만 에 배성웅 (...,0.0,416
3,20150801,20150801 23:31,20150801 23:31,4,1,"[ 롬 챔스 섬머 ] 스 베누 소닉붐 , 꼴찌 의 반격 개시 ! 그 동안 설움 을 ...",롤 챔스 섬머 2 경기 2 세트 는 기세 를 올린 스 베누 소닉붐 이 승리 를 가져...,헝그리앱,롤 챔스 섬머 2 경기 2 세트 는 기세 를 올린 스 베누 소닉붐 이 승리 를 가져...,0.0,0
4,20150801,20150801 21:58,20150801 21:58,5,1,"[ 롬 챔스 섬머 ] 스 베누 소닉붐 , 한 타 대승 기반 으로 2 세트 승리",롤 챔스 섬머 2 경기 2 세트 가 시작 됐 다 . 1 세트 때 진에어 그린 윙스 ...,헝그리앱,롤 챔스 섬머 2 경기 2 세트 가 시작 됐 다 . 1 세트 때 진에어 그린 윙스 ...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
51811,20200430,20200430 17:49,20200430 17:56,26,1,"[ LCK 승강 전 ] 다이브 받아치 면 곧 승리 ! 샌드박스 , 서라벌 에 1 세...",30 일 종각 LoL 파크 에서 열린 열린 2020 LCK 섬머 스플릿 승강 전 최...,인벤,30 일 종각 LoL 파크 에서 열린 열린 2020 LCK 섬머 스플릿 승강 전 최...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,"[ 롤 챔스 승강 전 ] 서라벌 - 샌드박스 , 베스트 라인업 으로 맞대결",LCK 2020 서머 최종 전 에서 대결 하 는 서라벌 게이밍 ( 위 ) 과 샌드박...,데일리e스포츠,마지막 승강 전 에서 승리 를 원하 는 서라벌 게이밍 과 샌드박스 게이밍 이 승강 ...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,"‘ 일방 적 인 전투 승리 ’… 샌드박스 게이밍 , 서라벌 게이밍 상대 로 1 세트 승리",30 일 ‘ 2020 우리은행 LCK Spring Split ’ 승강 전 최종진 출...,엑스포츠뉴스,30 일 ‘ 2020 우리은행 LCK Spring Split ’ 승강 전 최종진 출...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,"[ LCK 승강 전 ] 샌드박스 , 절묘 한 카운터 로 손쉽 게 기선 제압",[ OSEN = 고 용준 기자 ] LCK 최후 의 승강 전 에서 샌드박스 가 먼저 ...,OSEN,LCK 최후 의 승강 전 에서 샌드박스 가 먼저 웃 었 다 . 다이브 공세 를 절묘...,0.0,490


In [6]:
jamo_df = mecab_df.copy()
for column in TEXT_COLUMNS:
    jamo_df[column] = jamo_df[column].progress_apply(lambda x: jamo.j2hcj(jamo.h2j(x)) if x is not np.nan else x)
jamo_df.to_csv(tokenizing_dir / ('news-%s-%s-jamo.csv' % (DATA_TYPE, MECAB_TYPE)), index=False)
jamo_df

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
0,20150801,20150801 00:49,20150801 00:49,1,1,[ ㄹㅗㄹ ㅊㅐㅁㅅㅡ ㅎㅏㅅ ㅁㅐㅊㅣ ㄹㅣㅂㅠ ] ' ㅂㅜㄹㄹㅑㅇ ㅎㅏㄱㅅㅐㅇ ' ...,""" ㅌㅏㄹㅅㅓㄴ ㅎㅐㅆ ㅇㅓㅇㅛ ! """" ㅅㅜㄴㅡㅇ ㅁㅏㄴㅈㅓㅁ ! "" 2015 ㄴ...",인벤,""" ㅌㅏㄹㅅㅓㄴ ㅎㅐㅆ ㅇㅓㅇㅛ ! "". "" ㅅㅜㄴㅡㅇ ㅁㅏㄴㅈㅓㅁ ! "". 201...",0.0,198042
1,20150801,20150801 23:32,20150801 23:32,2,1,[ ㄹㅗㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ] ㅅㅡ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㄴㅠ ㅋㅡㄹㄹㅣㅇ...,ㄱㅡㅁㅇㅣㄹ ( 1 ㅇㅣㄹ ) ㄹㅗㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅅㅣㅈㅡㄴ ㅅㅏㅅㅏㅇ ㅅ...,헝그리앱,ㄱㅡㅁㅇㅣㄹ ( 1 ㅇㅣㄹ ) ㄹㅗㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅅㅣㅈㅡㄴ ㅅㅏㅅㅏㅇ ㅅ...,0.0,8657
2,20150801,20150801 20:10,20150801 20:10,3,1,[ ㄹㅗㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ] ㅂㅐㅇ ㅂㅐㅈㅜㄴㅅㅣㄱ ㅇㅣㄴㅌㅓㅂㅠ . ㅅㅗㄹ...,Q . ㅇㅗㄴㅡㄹ ㅅㅡㅇㄹㅣ ㅎㅏㄴ ㅅㅗㄱㅏㅁ ㅇㅡㄴ ㅇㅓㄸㅓㅎㄱㅔ ㄷㅚ ㄴㅏ ? ...,헝그리앱,Q . ㅇㅗㄴㅡㄹ ㅅㅡㅇㄹㅣ ㅎㅏㄴ ㅅㅗㄱㅏㅁ ㅇㅡㄴ ㅇㅓㄸㅓㅎㄱㅔ ㄷㅚ ㄴㅏ ? ...,0.0,416
3,20150801,20150801 23:31,20150801 23:31,4,1,"[ ㄹㅗㅁ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ] ㅅㅡ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ , ㄲㅗㄹㅉㅣ ㅇ...",ㄹㅗㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ 2 ㄱㅕㅇㄱㅣ 2 ㅅㅔㅌㅡ ㄴㅡㄴ ㄱㅣㅅㅔ ㄹㅡㄹ ㅇㅗ...,헝그리앱,ㄹㅗㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ 2 ㄱㅕㅇㄱㅣ 2 ㅅㅔㅌㅡ ㄴㅡㄴ ㄱㅣㅅㅔ ㄹㅡㄹ ㅇㅗ...,0.0,0
4,20150801,20150801 21:58,20150801 21:58,5,1,"[ ㄹㅗㅁ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ] ㅅㅡ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ , ㅎㅏㄴ ㅌㅏ ...",ㄹㅗㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ 2 ㄱㅕㅇㄱㅣ 2 ㅅㅔㅌㅡ ㄱㅏ ㅅㅣㅈㅏㄱ ㄷㅙㅆ ㄷㅏ...,헝그리앱,ㄹㅗㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ 2 ㄱㅕㅇㄱㅣ 2 ㅅㅔㅌㅡ ㄱㅏ ㅅㅣㅈㅏㄱ ㄷㅙㅆ ㄷㅏ...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
51811,20200430,20200430 17:49,20200430 17:56,26,1,[ LCK ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ] ㄷㅏㅇㅣㅂㅡ ㅂㅏㄷㅇㅏㅊㅣ ㅁㅕㄴ ㄱㅗㄷ ㅅㅡㅇㄹ...,30 ㅇㅣㄹ ㅈㅗㅇㄱㅏㄱ LoL ㅍㅏㅋㅡ ㅇㅔㅅㅓ ㅇㅕㄹㄹㅣㄴ ㅇㅕㄹㄹㅣㄴ 2020...,인벤,30 ㅇㅣㄹ ㅈㅗㅇㄱㅏㄱ LoL ㅍㅏㅋㅡ ㅇㅔㅅㅓ ㅇㅕㄹㄹㅣㄴ ㅇㅕㄹㄹㅣㄴ 2020...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,[ ㄹㅗㄹ ㅊㅐㅁㅅㅡ ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ] ㅅㅓㄹㅏㅂㅓㄹ - ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ...,LCK 2020 ㅅㅓㅁㅓ ㅊㅚㅈㅗㅇ ㅈㅓㄴ ㅇㅔㅅㅓ ㄷㅐㄱㅕㄹ ㅎㅏ ㄴㅡㄴ ㅅㅓㄹㅏ...,데일리e스포츠,ㅁㅏㅈㅣㅁㅏㄱ ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅇㅔㅅㅓ ㅅㅡㅇㄹㅣ ㄹㅡㄹ ㅇㅝㄴㅎㅏ ㄴㅡㄴ ㅅㅓ...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,‘ ㅇㅣㄹㅂㅏㅇ ㅈㅓㄱ ㅇㅣㄴ ㅈㅓㄴㅌㅜ ㅅㅡㅇㄹㅣ ’… ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㄱㅔㅇ...,30 ㅇㅣㄹ ‘ 2020 ㅇㅜㄹㅣㅇㅡㄴㅎㅐㅇ LCK Spring Split ’ ㅅㅡ...,엑스포츠뉴스,30 ㅇㅣㄹ ‘ 2020 ㅇㅜㄹㅣㅇㅡㄴㅎㅐㅇ LCK Spring Split ’ ㅅㅡ...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,"[ LCK ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ] ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ , ㅈㅓㄹㅁㅛ ㅎㅏㄴ ㅋㅏㅇㅜ...",[ OSEN = ㄱㅗ ㅇㅛㅇㅈㅜㄴ ㄱㅣㅈㅏ ] LCK ㅊㅚㅎㅜ ㅇㅢ ㅅㅡㅇㄱㅏㅇ ㅈ...,OSEN,LCK ㅊㅚㅎㅜ ㅇㅢ ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅇㅔㅅㅓ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㄱㅏ ㅁㅓㄴㅈ...,0.0,490


In [7]:
clean_df = mecab_df.copy()
for column in TEXT_COLUMNS:
    clean_df[column] = clean_df[column].progress_apply(lambda x: ' '.join(gensim.utils.simple_preprocess(x)) if x is not np.nan else x)
clean_df.to_csv(tokenizing_dir / ('news-%s-%s-clean.csv' % (DATA_TYPE, MECAB_TYPE)), index=False)
clean_df

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
0,20150801,20150801 00:49,20150801 00:49,1,1,챔스 매치 리뷰 불량 학생 노동현 수능 대박 스토리,탈선 어요 수능 만점 koo 타이거즈 kt 롤스터 정규 시즌 라운드 경기 펼쳐졌 모...,인벤,탈선 어요 수능 만점 koo 타이거즈 kt 롤스터 정규 시즌 라운드 경기 펼쳐졌 모...,0.0,198042
1,20150801,20150801 23:32,20150801 23:32,2,1,챔스 섬머 베누 소닉붐 클리어 사신 인터뷰 으로 승리 안겨 드리,금일 챔스 섬머 시즌 사상 베누 소닉붐 승리 기록 연패 마침내 승리 만큼 의미 감동,헝그리앱,금일 챔스 섬머 시즌 사상 베누 소닉붐 승리 기록 연패 마침내 승리 만큼 의미 감동...,0.0,8657
2,20150801,20150801 20:10,20150801 20:10,3,1,챔스 섬머 배준식 인터뷰 솔로 랭크 올리 멘탈 중요,오늘 승리 소감 어떻게 오늘 경기 오랜만 배성웅 이지훈 이지훈 오랜만 나왔 때문 으...,헝그리앱,오늘 승리 소감 어떻게 오늘 경기 오랜만 배성웅 이지훈 이지훈 오랜만 나왔 때문 으...,0.0,416
3,20150801,20150801 23:31,20150801 23:31,4,1,챔스 섬머 베누 소닉붐 꼴찌 반격 개시 동안 설움 설욕 대망 승리,챔스 섬머 경기 세트 기세 올린 베누 소닉붐 승리 가져갔 때문 진에어 그린 윙스 에...,헝그리앱,챔스 섬머 경기 세트 기세 올린 베누 소닉붐 승리 가져갔 때문 진에어 그린 윙스 에...,0.0,0
4,20150801,20150801 21:58,20150801 21:58,5,1,챔스 섬머 베누 소닉붐 대승 기반 으로 세트 승리,챔스 섬머 경기 세트 시작 세트 진에어 그린 윙스 에게 억눌린 베누 소닉붐 세트 미...,헝그리앱,챔스 섬머 경기 세트 시작 세트 진에어 그린 윙스 에게 억눌린 베누 소닉붐 세트 미...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
51811,20200430,20200430 17:49,20200430 17:56,26,1,lck 승강 다이브 받아치 승리 샌드박스 서라벌 세트 선승,종각 lol 파크 에서 열린 열린 lck 섬머 스플릿 승강 최종 세트 에서 샌드박스...,인벤,종각 lol 파크 에서 열린 열린 lck 섬머 스플릿 승강 최종 세트 에서 샌드박스...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,챔스 승강 서라벌 샌드박스 베스트 라인업 으로 맞대결,lck 서머 최종 에서 대결 서라벌 게이밍 샌드박스 게이밍 사진 라이엇 게임즈 제공...,데일리e스포츠,마지막 승강 에서 승리 원하 서라벌 게이밍 샌드박스 게이밍 승강 내내 기용 베스트 ...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,일방 전투 승리 샌드박스 게이밍 서라벌 게이밍 상대 세트 승리,우리은행 lck spring split 승강 최종진 출전 에서 샌드박스 게이밍 서라...,엑스포츠뉴스,우리은행 lck spring split 승강 최종진 출전 에서 샌드박스 게이밍 서라...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,lck 승강 샌드박스 절묘 카운터 손쉽 기선 제압,osen 용준 기자 lck 최후 승강 에서 샌드박스 먼저 다이브 공세 절묘 받아치 ...,OSEN,lck 최후 승강 에서 샌드박스 먼저 다이브 공세 절묘 받아치 면서 흐름 샌드박스 ...,0.0,490


In [8]:
clean_jamo_df = clean_df.copy()
for column in TEXT_COLUMNS:
    clean_jamo_df[column] = clean_jamo_df[column].progress_apply(lambda x: jamo.j2hcj(jamo.h2j(x)) if x is not np.nan else x)
clean_jamo_df.to_csv(tokenizing_dir / ('news-%s-%s-clean-jamo.csv' % (DATA_TYPE, MECAB_TYPE)), index=False)
clean_jamo_df

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
0,20150801,20150801 00:49,20150801 00:49,1,1,ㅊㅐㅁㅅㅡ ㅁㅐㅊㅣ ㄹㅣㅂㅠ ㅂㅜㄹㄹㅑㅇ ㅎㅏㄱㅅㅐㅇ ㄴㅗㄷㅗㅇㅎㅕㄴ ㅅㅜㄴㅡㅇ ㄷ...,ㅌㅏㄹㅅㅓㄴ ㅇㅓㅇㅛ ㅅㅜㄴㅡㅇ ㅁㅏㄴㅈㅓㅁ koo ㅌㅏㅇㅣㄱㅓㅈㅡ kt ㄹㅗㄹㅅㅡ...,인벤,ㅌㅏㄹㅅㅓㄴ ㅇㅓㅇㅛ ㅅㅜㄴㅡㅇ ㅁㅏㄴㅈㅓㅁ koo ㅌㅏㅇㅣㄱㅓㅈㅡ kt ㄹㅗㄹㅅㅡ...,0.0,198042
1,20150801,20150801 23:32,20150801 23:32,2,1,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅋㅡㄹㄹㅣㅇㅓ ㅅㅏㅅㅣㄴ ㅇㅣㄴㅌㅓㅂ...,ㄱㅡㅁㅇㅣㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅅㅣㅈㅡㄴ ㅅㅏㅅㅏㅇ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅅ...,헝그리앱,ㄱㅡㅁㅇㅣㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅅㅣㅈㅡㄴ ㅅㅏㅅㅏㅇ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅅ...,0.0,8657
2,20150801,20150801 20:10,20150801 20:10,3,1,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅂㅐㅈㅜㄴㅅㅣㄱ ㅇㅣㄴㅌㅓㅂㅠ ㅅㅗㄹㄹㅗ ㄹㅐㅇㅋㅡ ㅇㅗㄹㄹㅣ...,ㅇㅗㄴㅡㄹ ㅅㅡㅇㄹㅣ ㅅㅗㄱㅏㅁ ㅇㅓㄸㅓㅎㄱㅔ ㅇㅗㄴㅡㄹ ㄱㅕㅇㄱㅣ ㅇㅗㄹㅐㄴㅁㅏㄴ...,헝그리앱,ㅇㅗㄴㅡㄹ ㅅㅡㅇㄹㅣ ㅅㅗㄱㅏㅁ ㅇㅓㄸㅓㅎㄱㅔ ㅇㅗㄴㅡㄹ ㄱㅕㅇㄱㅣ ㅇㅗㄹㅐㄴㅁㅏㄴ...,0.0,416
3,20150801,20150801 23:31,20150801 23:31,4,1,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㄲㅗㄹㅉㅣ ㅂㅏㄴㄱㅕㄱ ㄱㅐㅅㅣ ㄷㅗ...,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㄱㅣㅅㅔ ㅇㅗㄹㄹㅣㄴ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂ...,헝그리앱,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㄱㅣㅅㅔ ㅇㅗㄹㄹㅣㄴ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂ...,0.0,0
4,20150801,20150801 21:58,20150801 21:58,5,1,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㄷㅐㅅㅡㅇ ㄱㅣㅂㅏㄴ ㅇㅡㄹㅗ ㅅㅔㅌ...,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㅅㅣㅈㅏㄱ ㅅㅔㅌㅡ ㅈㅣㄴㅇㅔㅇㅓ ㄱㅡㄹㅣ...,헝그리앱,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㅅㅣㅈㅏㄱ ㅅㅔㅌㅡ ㅈㅣㄴㅇㅔㅇㅓ ㄱㅡㄹㅣ...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
51811,20200430,20200430 17:49,20200430 17:56,26,1,lck ㅅㅡㅇㄱㅏㅇ ㄷㅏㅇㅣㅂㅡ ㅂㅏㄷㅇㅏㅊㅣ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅅㅓㄹ...,ㅈㅗㅇㄱㅏㄱ lol ㅍㅏㅋㅡ ㅇㅔㅅㅓ ㅇㅕㄹㄹㅣㄴ ㅇㅕㄹㄹㅣㄴ lck ㅅㅓㅁㅁㅓ ㅅ...,인벤,ㅈㅗㅇㄱㅏㄱ lol ㅍㅏㅋㅡ ㅇㅔㅅㅓ ㅇㅕㄹㄹㅣㄴ ㅇㅕㄹㄹㅣㄴ lck ㅅㅓㅁㅁㅓ ㅅ...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,ㅊㅐㅁㅅㅡ ㅅㅡㅇㄱㅏㅇ ㅅㅓㄹㅏㅂㅓㄹ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅂㅔㅅㅡㅌㅡ ㄹㅏㅇㅣㄴㅇㅓ...,lck ㅅㅓㅁㅓ ㅊㅚㅈㅗㅇ ㅇㅔㅅㅓ ㄷㅐㄱㅕㄹ ㅅㅓㄹㅏㅂㅓㄹ ㄱㅔㅇㅣㅁㅣㅇ ㅅㅐㄴㄷ...,데일리e스포츠,ㅁㅏㅈㅣㅁㅏㄱ ㅅㅡㅇㄱㅏㅇ ㅇㅔㅅㅓ ㅅㅡㅇㄹㅣ ㅇㅝㄴㅎㅏ ㅅㅓㄹㅏㅂㅓㄹ ㄱㅔㅇㅣㅁㅣ...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,ㅇㅣㄹㅂㅏㅇ ㅈㅓㄴㅌㅜ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㄱㅔㅇㅣㅁㅣㅇ ㅅㅓㄹㅏㅂㅓㄹ ...,ㅇㅜㄹㅣㅇㅡㄴㅎㅐㅇ lck spring split ㅅㅡㅇㄱㅏㅇ ㅊㅚㅈㅗㅇㅈㅣㄴ ㅊㅜ...,엑스포츠뉴스,ㅇㅜㄹㅣㅇㅡㄴㅎㅐㅇ lck spring split ㅅㅡㅇㄱㅏㅇ ㅊㅚㅈㅗㅇㅈㅣㄴ ㅊㅜ...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,lck ㅅㅡㅇㄱㅏㅇ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅈㅓㄹㅁㅛ ㅋㅏㅇㅜㄴㅌㅓ ㅅㅗㄴㅅㅟㅂ ㄱㅣㅅ...,osen ㅇㅛㅇㅈㅜㄴ ㄱㅣㅈㅏ lck ㅊㅚㅎㅜ ㅅㅡㅇㄱㅏㅇ ㅇㅔㅅㅓ ㅅㅐㄴㄷㅡㅂㅏㄱ...,OSEN,lck ㅊㅚㅎㅜ ㅅㅡㅇㄱㅏㅇ ㅇㅔㅅㅓ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅁㅓㄴㅈㅓ ㄷㅏㅇㅣㅂㅡ ㄱ...,0.0,490
